**The idea is:**

 - Feature reduction with PCA
 - Data transformation (log, hot encoding, nan)
 - Test different regression models

**Things found:**

- Applying log transformation really increases the accuracy.
- Using PCA with 36 components makes the learning and testing much (much much) faster.
- Removing columns with more than 1000 NaNs gives better result than applying "mean" to them.
- There are outliers. Instead of removing them, using Huber seems to provide a good result. Huber is a model robust to outliers.

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.decomposition import PCA
from sklearn.preprocessing import Imputer
from sklearn.model_selection import KFold
from sklearn import linear_model
from sklearn.metrics import make_scorer
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn import svm
from sklearn.metrics import r2_score
from sklearn.ensemble import AdaBoostRegressor
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt
import tflearn
import tensorflow as tf
import seaborn
import warnings
warnings.filterwarnings('ignore')

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

sample_submission.csv
test.csv
train.csv



## Data Load ##

I mix data and test to manipulate all the data just once. SalePrice is extracted to its own variable "labels". Finally, SalesPrice is remove from data.

In [2]:
train = pd.read_csv('../input/train.csv')
labels=train["SalePrice"]
test = pd.read_csv('../input/test.csv')
data = pd.concat([train,test],ignore_index=True)
data = data.drop("SalePrice", 1)
ids = test["Id"]

In [3]:
train.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [4]:
# Count the number of rows in train
train.shape[0]

1460

In [6]:
# Count the number of rows in total
data.shape[0]

2919

In [5]:
# Count the number of NaNs each column has.
nans=pd.isnull(data).sum()
nans[nans>0]

Alley           2721
BsmtCond          82
BsmtExposure      82
BsmtFinSF1         1
BsmtFinSF2         1
BsmtFinType1      79
BsmtFinType2      80
BsmtFullBath       2
BsmtHalfBath       2
BsmtQual          81
BsmtUnfSF          1
Electrical         1
Exterior1st        1
Exterior2nd        1
Fence           2348
FireplaceQu     1420
Functional         2
GarageArea         1
GarageCars         1
GarageCond       159
GarageFinish     159
GarageQual       159
GarageType       157
GarageYrBlt      159
KitchenQual        1
LotFrontage      486
MSZoning           4
MasVnrArea        23
MasVnrType        24
MiscFeature     2814
PoolQC          2909
SaleType           1
TotalBsmtSF        1
Utilities          2
dtype: int64

In [7]:
# Remove id and columns with more than a thousand missing values
data=data.drop("Id", 1)
data=data.drop("Alley", 1)
data=data.drop("Fence", 1)
data=data.drop("MiscFeature", 1)
data=data.drop("PoolQC", 1)
data=data.drop("FireplaceQu", 1)

In [8]:
# Count the column types
data.dtypes.value_counts()

object     38
int64      25
float64    11
dtype: int64

## Data Manipulation ##

- Apply hot encoding, convert categorical variable into dummy/indicator variables.
- Fill NaN with median for that column.
- Log transformation.
- Change -inf to 0.

In [9]:
all_columns = data.columns.values
non_categorical = ["LotFrontage", "LotArea", "MasVnrArea", "BsmtFinSF1", 
                   "BsmtFinSF2", "BsmtUnfSF", "TotalBsmtSF", "1stFlrSF", 
                   "2ndFlrSF", "LowQualFinSF", "GrLivArea", "GarageArea", 
                   "WoodDeckSF", "OpenPorchSF", "EnclosedPorch", "3SsnPorch", 
                   "ScreenPorch","PoolArea", "MiscVal"]

categorical = [value for value in all_columns if value not in non_categorical]

In [10]:
# One Hot Encoding and nan transformation
data = pd.get_dummies(data)

imp = Imputer(missing_values='NaN', strategy='most_frequent', axis=0)
data = imp.fit_transform(data)

# Log transformation
data = np.log(data)
labels = np.log(labels)

# Change -inf to 0 again
data[data==-np.inf]=0

## Feature reduction ##

There are many features, so I am going to use PCA to reduce them. The idea is to start with n_components = number of columns. Then select the number of components that add up to 1 variance_ratio.

In [11]:
pca = PCA(whiten=True)
pca.fit(data)
variance = pd.DataFrame(pca.explained_variance_ratio_)
np.cumsum(pca.explained_variance_ratio_)

array([ 0.2248857 ,  0.40281429,  0.52425789,  0.62418823,  0.69580422,
        0.75944463,  0.8116806 ,  0.85647038,  0.89178708,  0.92273755,
        0.94898868,  0.95842727,  0.96637545,  0.97380464,  0.97971901,
        0.98501952,  0.98918839,  0.99199181,  0.99386559,  0.99520919,
        0.99611479,  0.99695667,  0.99771023,  0.99842564,  0.9989402 ,
        0.99933882,  0.99959949,  0.99978254,  0.99988174,  0.99993998,
        0.99998599,  0.99999658,  0.99999871,  0.99999943,  0.99999999,
        1.        ,  1.        ,  1.        ,  1.        ,  1.        ,
        1.        ,  1.        ,  1.        ,  1.        ,  1.        ,
        1.        ,  1.        ,  1.        ,  1.        ,  1.        ,
        1.        ,  1.        ,  1.        ,  1.        ,  1.        ,
        1.        ,  1.        ,  1.        ,  1.        ,  1.        ,
        1.        ,  1.        ,  1.        ,  1.        ,  1.        ,
        1.        ,  1.        ,  1.        ,  1.        ,  1.  

In [12]:
pca = PCA(n_components=36,whiten=True)
pca = pca.fit(data)
dataPCA = pca.transform(data)

## Data Model Selection ##

Simple test to run multiple models against our data. First, with raw features. No PCA.

In [13]:
# Split traing and test
train = data[:1460]
test = data[1460:]

In [ ]:
# R2 Score

def lets_try(train,labels):
    results={}
    def test_model(clf):
        
        cv = KFold(n_splits=5,shuffle=True,random_state=45)
        r2 = make_scorer(r2_score)
        r2_val_score = cross_val_score(clf, train, labels, cv=cv,scoring=r2)
        scores=[r2_val_score.mean()]
        return scores

    clf = linear_model.LinearRegression()
    results["Linear"]=test_model(clf)
    
    clf = linear_model.Ridge()
    results["Ridge"]=test_model(clf)
    
    clf = linear_model.BayesianRidge()
    results["Bayesian Ridge"]=test_model(clf)
    
    clf = linear_model.HuberRegressor()
    results["Hubber"]=test_model(clf)
    
    clf = linear_model.Lasso(alpha=1e-4)
    results["Lasso"]=test_model(clf)
    
    clf = BaggingRegressor()
    results["Bagging"]=test_model(clf)
    
    clf = RandomForestRegressor()
    results["RandomForest"]=test_model(clf)
    
    clf = AdaBoostRegressor()
    results["AdaBoost"]=test_model(clf)
    
    clf = svm.SVR()
    results["SVM RBF"]=test_model(clf)
    
    clf = svm.SVR(kernel="linear")
    results["SVM Linear"]=test_model(clf)
    
    results = pd.DataFrame.from_dict(results,orient='index')
    results.columns=["R Square Score"] 
    results=results.sort(columns=["R Square Score"],ascending=False)
    results.plot(kind="bar",title="Model Scores")
    axes = plt.gca()
    axes.set_ylim([0.5,1])
    return results

lets_try(train,labels)

Now, let's try the same but using data with PCA applied.

In [14]:
# Split traing and test
train = dataPCA[:1460]
test = dataPCA[1460:]

lets_try(train,labels)

NameError: name 'lets_try' is not defined

In [ ]:
cv = KFold(n_splits=5,shuffle=True,random_state=45)

parameters = {'alpha': [1000,100,10],
              'epsilon' : [1.2,1.25,1.50],
              'tol' : [1e-10]}

clf = linear_model.HuberRegressor()
r2 = make_scorer(r2_score)
grid_obj = GridSearchCV(clf, parameters, cv=cv,scoring=r2)
grid_fit = grid_obj.fit(train, labels)
best_clf = grid_fit.best_estimator_ 

best_clf.fit(train,labels)

Simple Neural Network
---------------------

Now I am going to try a simple neural network, to see if i can improve the result.

In [15]:
# Shape the labels
labels_nl = labels
labels_nl = labels_nl.reshape(-1,1)

In [43]:
tf.reset_default_graph()
r2 = tflearn.R2()
net = tflearn.input_data(shape=[None, train.shape[1]])
net = tflearn.fully_connected(net, 30, activation='linear')
net = tflearn.fully_connected(net, 30, activation='linear')
net = tflearn.fully_connected(net, 20, activation='linear')
net = tflearn.fully_connected(net, 20, activation='linear')
net = tflearn.fully_connected(net, 10, activation='linear')
net = tflearn.fully_connected(net, 10, activation='linear')
net = tflearn.fully_connected(net, 1, activation='linear')
sgd = tflearn.SGD(learning_rate=0.1, lr_decay=0.01, decay_step=100)
net = tflearn.regression(net, optimizer=sgd, loss='mean_square',metric=r2)
model = tflearn.DNN(net)

In [44]:
model.fit(train, labels_nl,show_metric=True,validation_set=0.2,shuffle=True,n_epoch=50)

---------------------------------
Run id: XYUAEX
Log directory: /tmp/tflearn_logs/
INFO:tensorflow:Summary name StandardError/ (raw) is illegal; using StandardError/__raw_ instead.
---------------------------------
Training samples: 1168
Validation samples: 292
--
Training Step: 1  | total loss: 132.00549 | time: 0.037s
| SGD | epoch: 001 | loss: 132.00549 - R2: 0.0000 -- iter: 0020/1168
Training Step: 2  | total loss: 142.06914 | time: 0.044s
| SGD | epoch: 001 | loss: 142.06914 - R2: 0.0000 -- iter: 0040/1168
Training Step: 3  | total loss: 143.14580 | time: 0.049s
| SGD | epoch: 001 | loss: 143.14580 - R2: 0.0001 -- iter: 0060/1168
Training Step: 4  | total loss: 141.59778 | time: 0.057s
| SGD | epoch: 001 | loss: 141.59778 - R2: 0.0001 -- iter: 0080/1168
Training Step: 5  | total loss: 141.59778 | time: 0.064s
| SGD | epoch: 001 | loss: 141.59778 - R2: 0.0001 -- iter: 0100/1168
Training Step: 6  | total loss: 141.35030 | time: 0.069s
| SGD | epoch: 001 | loss: 141.35030 - R2: 0.000

Training Step: 57  | total loss: 91.51981 | time: 0.291s
| SGD | epoch: 001 | loss: 91.51981 - R2: 0.0426 -- iter: 1140/1168
Training Step: 58  | total loss: 89.66460 | time: 0.294s
| SGD | epoch: 001 | loss: 89.66460 - R2: 0.0460 -- iter: 1160/1168
Training Step: 59  | total loss: 88.88686 | time: 1.316s
| SGD | epoch: 001 | loss: 88.88686 - R2: 0.0477 | val_loss: 81.62693 - val_acc: 0.0617 -- iter: 1168/1168
--
Training Step: 60  | total loss: 88.08608 | time: 0.054s
| SGD | epoch: 002 | loss: 88.08608 - R2: 0.0495 -- iter: 0020/1168
Training Step: 61  | total loss: 86.81692 | time: 0.063s
| SGD | epoch: 002 | loss: 86.81692 - R2: 0.0515 -- iter: 0040/1168
Training Step: 62  | total loss: 85.67319 | time: 0.068s
| SGD | epoch: 002 | loss: 85.67319 - R2: 0.0536 -- iter: 0060/1168
Training Step: 63  | total loss: 84.95834 | time: 0.072s
| SGD | epoch: 002 | loss: 84.95834 - R2: 0.0554 -- iter: 0080/1168
Training Step: 64  | total loss: 83.99094 | time: 0.076s
| SGD | epoch: 002 | loss:

Training Step: 115  | total loss: 27.83018 | time: 0.367s
| SGD | epoch: 002 | loss: 27.83018 - R2: 0.3305 -- iter: 1120/1168
Training Step: 116  | total loss: 26.70437 | time: 0.371s
| SGD | epoch: 002 | loss: 26.70437 - R2: 0.3413 -- iter: 1140/1168
Training Step: 117  | total loss: 26.70437 | time: 0.375s
| SGD | epoch: 002 | loss: 26.70437 - R2: 0.3413 -- iter: 1160/1168
Training Step: 118  | total loss: 25.62698 | time: 1.380s
| SGD | epoch: 002 | loss: 25.62698 - R2: 0.3523 | val_loss: 13.28438 - val_acc: 0.4872 -- iter: 1168/1168
--
Training Step: 119  | total loss: 24.54867 | time: 0.021s
| SGD | epoch: 003 | loss: 24.54867 - R2: 0.3765 -- iter: 0020/1168
Training Step: 120  | total loss: 22.19878 | time: 0.024s
| SGD | epoch: 003 | loss: 22.19878 - R2: 0.3900 -- iter: 0040/1168
Training Step: 121  | total loss: 21.05578 | time: 0.027s
| SGD | epoch: 003 | loss: 21.05578 - R2: 0.4038 -- iter: 0060/1168
Training Step: 122  | total loss: 20.05187 | time: 0.031s
| SGD | epoch: 003

Training Step: 173  | total loss: 0.21417 | time: 0.245s
| SGD | epoch: 003 | loss: 0.21417 - R2: 0.9901 -- iter: 1100/1168
Training Step: 174  | total loss: 0.17586 | time: 0.248s
| SGD | epoch: 003 | loss: 0.17586 - R2: 0.9920 -- iter: 1120/1168
Training Step: 175  | total loss: 0.17586 | time: 0.251s
| SGD | epoch: 003 | loss: 0.17586 - R2: 0.9920 -- iter: 1140/1168
Training Step: 176  | total loss: 0.14538 | time: 0.253s
| SGD | epoch: 003 | loss: 0.14538 - R2: 0.9931 -- iter: 1160/1168
Training Step: 177  | total loss: 0.13345 | time: 1.266s
| SGD | epoch: 003 | loss: 0.13345 - R2: 0.9943 | val_loss: 0.03061 - val_acc: 1.0001 -- iter: 1168/1168
--
Training Step: 178  | total loss: 0.12247 | time: 0.040s
| SGD | epoch: 004 | loss: 0.12247 - R2: 0.9941 -- iter: 0020/1168
Training Step: 179  | total loss: 0.12247 | time: 0.043s
| SGD | epoch: 004 | loss: 0.12247 - R2: 0.9941 -- iter: 0040/1168
Training Step: 180  | total loss: 0.10866 | time: 0.056s
| SGD | epoch: 004 | loss: 0.10866

Training Step: 231  | total loss: 0.02748 | time: 0.274s
| SGD | epoch: 004 | loss: 0.02748 - R2: 0.9996 -- iter: 1080/1168
Training Step: 232  | total loss: 0.02516 | time: 0.277s
| SGD | epoch: 004 | loss: 0.02516 - R2: 0.9998 -- iter: 1100/1168
Training Step: 233  | total loss: 0.02336 | time: 0.281s
| SGD | epoch: 004 | loss: 0.02336 - R2: 1.0000 -- iter: 1120/1168
Training Step: 234  | total loss: 0.02336 | time: 0.300s
| SGD | epoch: 004 | loss: 0.02336 - R2: 1.0000 -- iter: 1140/1168
Training Step: 235  | total loss: 0.02129 | time: 0.305s
| SGD | epoch: 004 | loss: 0.02129 - R2: 0.9997 -- iter: 1160/1168
Training Step: 236  | total loss: 0.02129 | time: 1.312s
| SGD | epoch: 004 | loss: 0.02129 - R2: 0.9997 | val_loss: 0.03318 - val_acc: 0.9993 -- iter: 1168/1168
--
Training Step: 237  | total loss: 0.02162 | time: 0.070s
| SGD | epoch: 005 | loss: 0.02162 - R2: 1.0000 -- iter: 0020/1168
Training Step: 238  | total loss: 0.02073 | time: 0.101s
| SGD | epoch: 005 | loss: 0.02073

...

Training Step: 659  | total loss: 0.16345 | time: 0.115s
| SGD | epoch: 035 | loss: 0.16345 - R2: 0.9995 -- iter: 0832/1168
Training Step: 660  | total loss: 0.15541 | time: 0.146s
| SGD | epoch: 035 | loss: 0.15541 - R2: 0.9997 -- iter: 0896/1168
Training Step: 661  | total loss: 0.14655 | time: 0.152s
| SGD | epoch: 035 | loss: 0.14655 - R2: 1.0000 -- iter: 0960/1168
Training Step: 662  | total loss: 0.14665 | time: 0.158s
| SGD | epoch: 035 | loss: 0.14665 - R2: 1.0015 -- iter: 1024/1168
Training Step: 663  | total loss: 0.14745 | time: 0.164s
| SGD | epoch: 035 | loss: 0.14745 - R2: 1.0017 -- iter: 1088/1168
Training Step: 664  | total loss: 0.14695 | time: 0.169s
| SGD | epoch: 035 | loss: 0.14695 - R2: 1.0011 -- iter: 1152/1168
Training Step: 665  | total loss: 0.14503 | time: 1.175s
| SGD | epoch: 035 | loss: 0.14503 - R2: 1.0003 | val_loss: 0.16063 - val_acc: 0.9945 -- iter: 1168/1168
--
Training Step: 666  | total loss: 0.14503 | time: 0.049s
| SGD | epoch: 036 | loss: 0.14503

Training Step: 717  | total loss: 0.15614 | time: 0.121s
| SGD | epoch: 038 | loss: 0.15614 - R2: 0.9982 -- iter: 0896/1168
Training Step: 718  | total loss: 0.15888 | time: 0.127s
| SGD | epoch: 038 | loss: 0.15888 - R2: 0.9985 -- iter: 0960/1168
Training Step: 719  | total loss: 0.15909 | time: 0.131s
| SGD | epoch: 038 | loss: 0.15909 - R2: 0.9968 -- iter: 1024/1168
Training Step: 720  | total loss: 0.15909 | time: 0.142s
| SGD | epoch: 038 | loss: 0.15909 - R2: 0.9968 -- iter: 1088/1168
Training Step: 721  | total loss: 0.15928 | time: 0.145s
| SGD | epoch: 038 | loss: 0.15928 - R2: 0.9952 -- iter: 1152/1168
Training Step: 722  | total loss: 0.16082 | time: 1.152s
| SGD | epoch: 038 | loss: 0.16082 - R2: 0.9954 | val_loss: 0.16063 - val_acc: 0.9945 -- iter: 1168/1168
--
Training Step: 723  | total loss: 0.16309 | time: 0.049s
| SGD | epoch: 039 | loss: 0.16309 - R2: 0.9967 -- iter: 0064/1168
Training Step: 724  | total loss: 0.15854 | time: 0.052s
| SGD | epoch: 039 | loss: 0.15854

Training Step: 775  | total loss: 0.15057 | time: 0.138s
| SGD | epoch: 041 | loss: 0.15057 - R2: 1.0007 -- iter: 0960/1168
Training Step: 776  | total loss: 0.14815 | time: 0.142s
| SGD | epoch: 041 | loss: 0.14815 - R2: 1.0002 -- iter: 1024/1168
Training Step: 777  | total loss: 0.14815 | time: 0.148s
| SGD | epoch: 041 | loss: 0.14815 - R2: 1.0002 -- iter: 1088/1168
Training Step: 778  | total loss: 0.15309 | time: 0.151s
| SGD | epoch: 041 | loss: 0.15309 - R2: 0.9997 -- iter: 1152/1168
Training Step: 779  | total loss: 0.14835 | time: 1.158s
| SGD | epoch: 041 | loss: 0.14835 - R2: 1.0014 | val_loss: 0.16063 - val_acc: 0.9945 -- iter: 1168/1168
--
Training Step: 780  | total loss: 0.14835 | time: 0.041s
| SGD | epoch: 042 | loss: 0.14835 - R2: 1.0014 -- iter: 0064/1168
Training Step: 781  | total loss: 0.14394 | time: 0.044s
| SGD | epoch: 042 | loss: 0.14394 - R2: 1.0033 -- iter: 0128/1168
Training Step: 782  | total loss: 0.14938 | time: 0.048s
| SGD | epoch: 042 | loss: 0.14938

Training Step: 833  | total loss: 0.15571 | time: 0.139s
| SGD | epoch: 044 | loss: 0.15571 - R2: 0.9966 -- iter: 1024/1168
Training Step: 834  | total loss: 0.15870 | time: 0.144s
| SGD | epoch: 044 | loss: 0.15870 - R2: 0.9982 -- iter: 1088/1168
Training Step: 835  | total loss: 0.15817 | time: 0.146s
| SGD | epoch: 044 | loss: 0.15817 - R2: 0.9979 -- iter: 1152/1168
Training Step: 836  | total loss: 0.16059 | time: 1.152s
| SGD | epoch: 044 | loss: 0.16059 - R2: 0.9986 | val_loss: 0.16063 - val_acc: 0.9945 -- iter: 1168/1168
--
Training Step: 837  | total loss: 0.15755 | time: 0.056s
| SGD | epoch: 045 | loss: 0.15755 - R2: 0.9995 -- iter: 0064/1168
Training Step: 838  | total loss: 0.15985 | time: 0.070s
| SGD | epoch: 045 | loss: 0.15985 - R2: 0.9991 -- iter: 0128/1168
Training Step: 839  | total loss: 0.16316 | time: 0.073s
| SGD | epoch: 045 | loss: 0.16316 - R2: 0.9997 -- iter: 0192/1168
Training Step: 840  | total loss: 0.15898 | time: 0.075s
| SGD | epoch: 045 | loss: 0.15898

Training Step: 891  | total loss: 0.16434 | time: 0.109s
| SGD | epoch: 047 | loss: 0.16434 - R2: 0.9996 -- iter: 1088/1168
Training Step: 892  | total loss: 0.16508 | time: 0.115s
| SGD | epoch: 047 | loss: 0.16508 - R2: 0.9992 -- iter: 1152/1168
Training Step: 893  | total loss: 0.16635 | time: 1.122s
| SGD | epoch: 047 | loss: 0.16635 - R2: 0.9988 | val_loss: 0.16063 - val_acc: 0.9945 -- iter: 1168/1168
--
Training Step: 894  | total loss: 0.15601 | time: 0.005s
| SGD | epoch: 048 | loss: 0.15601 - R2: 1.0004 -- iter: 0064/1168
Training Step: 895  | total loss: 0.16165 | time: 0.010s
| SGD | epoch: 048 | loss: 0.16165 - R2: 0.9987 -- iter: 0128/1168
Training Step: 896  | total loss: 0.16165 | time: 0.014s
| SGD | epoch: 048 | loss: 0.16165 - R2: 0.9987 -- iter: 0192/1168
Training Step: 897  | total loss: 0.16729 | time: 0.017s
| SGD | epoch: 048 | loss: 0.16729 - R2: 0.9991 -- iter: 0256/1168
Training Step: 898  | total loss: 0.16823 | time: 0.022s
| SGD | epoch: 048 | loss: 0.16823

In [ ]:
# Make predictions

predictions_huber = best_clf.predict(test)
predictions_DNN = model.predict(test)
predictions_huber = np.exp(predictions_huber)
predictions_DNN = np.exp(predictions_DNN)
predictions_DNN = predictions_DNN.reshape(-1,)

sub = pd.DataFrame({
        "Id": ids,
        "SalePrice": predictions_DNN
    })

sub.to_csv("prices_submission.csv", index=False)
#print(sub)